<h1><center>Einführung in die Neuroinformatik - Übungsblatt 5</center></h1>
<h2><center>Dominik Authaler, Marco Deuscher und Carolin Schindler</center></h2>
<h2><center>Gruppe Q</center></h2>
<h2><center>Juni 2019</center></h2>

## Aufgabe 1: 

$$
\begin{array}{ccc}
\hline
\text{Schritt} & \text{Berechnung} & \text{Benötigte Variablen} & \text{Phase} \\
\hline
1 & u_1^{(1)} & x_1, w_{11}^{(1)}, x_2, w_{21}^{(1)}& \text{I}\\
2 & y_1^{(1)}& u_1^{(1)}, b_1^{(1)}& \text{I}\\
3 & u_1^{(2)}& y_1^{(1)}, w_{11}^{(2)}, y_2^{(1)}, w_{21}^{(2)}& \text{I}\\
4 & y_1^{(2)}& u_1^{(2)}, b_1^{(2)}& \text{I}\\
5 & \delta_1^{(2)}& y_1^{(2)}, T, u_1^{(2)}& \text{II}\\
6 & \delta_1^{(1)}& w_{11}^{(2)}, \delta_1^{(2)}, u_1^{(1)}& \text{III}\\
7 & w_{11}^{(1)}& x_1, x_2, \eta, \delta_1^{(1)} & \text{IV}\\
\hline
\end{array}
$$

## Aufgabe 2: Backpropagation selbst implementieren

In [101]:
%matplotlib widget
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(42)

### Funktionen

In [111]:
def tanh_deriv(inp):
    return 1 / (np.cosh(inp)**2)

def forward(inpX, w1, w2, b1, b2):
    '''
    Perform a forward step of the network. For the transfer function in the hidden layer, use tanh.
    
    Parameters
    ----------
    inpX : data matrix
        input matrix, shaped as: samples x dimensions
    w1 : matrix
        weight matrix between input and hidden neurons
    w2 : matrix
        weight matrix between hidden and output neurons
    b1 : vector
        bias vector for the hidden neurons
    b2 : vector
        bias vector for the output neurons
    '''
    
    u1 = np.add(np.dot(inpX, w1), b1)
    y1 = np.tanh(u1)
    y2 = np.asscalar(np.add(np.dot(y1, w2), b2))
    
    #print(inpX.shape)
    #print(w1.shape)
    #print(w2.shape)
    #print(b1.shape)
    #print(b2.shape)
    
    #print("Res: ")
    #print(y1.shape)
    
    return u1, y1, y2

def initialize_weights(inpDim, hiddenNeurons, outDim):
    '''
    Initialize the weight matrix based on input Dimension, amount of hidden neurons and output dimension.
    The range for the initial weights is given by [-.5; .5].
    
    Parameters
    ----------
    inpDim : int
        Number of input neurons
    hiddenNeurons : int
        Number of hidden neurons
    outDim : int
        Number of output neurons
    weights : list
        List containing the weights and biases in the following order: [w1, w2, b1, b2]
    '''
    
    w1 = np.subtract(np.random.rand(inpDim, hiddenNeurons), 0.5)
    
    w2 = np.subtract(np.random.rand(hiddenNeurons, outDim), 0.5)
    
    b1 = np.subtract(np.random.rand(1, hiddenNeurons), 0.5)
    
    b2 = np.subtract(np.random.rand(1, outDim), 0.5)
    
    #print(w1.shape)
    #print(w2.shape)
    #print(b1.shape)
    #print(b2.shape)
    
    weights = [w1, w2, b1, b2];
    
    return weights

def prop_error(T, y2, w2, transDiff_u1):
    '''
    Calculation of the error of the network
    
    Parameters
    ----------
    T : float
        teaching signal of the current sample
    y2 : float
        output of the last neuron
    w2 : data matrix
        weight matrix between hidden and output layer
    transDiff_u1 : vector
        differential of the transfer function used on u1
    '''
    
    # Ableitung von Transferfunktion f(x) = x ist 1
    delta2 = (T-y2);
    
    delta1 = w2.transpose()*delta2*transDiff_u1
    
    return delta1, delta2

def training(hiddenNeurons, lernRate, inpX_T, inpDim, epoch):
    '''
    Train the neural network. 
    
    Parameters
    ----------
    hiddenNeurons : int
        Number of hidden Neurons
    lernRate : float
        Lernrate \eta
    inpX_T : combined matrix of the inputData and the teaching signal
        input data and shaped as: samples x dimensions 
    inpDim : dimension of the inputData
    epoch : int
        number of training epochs
    '''
    
    #print(inpX_T.shape)
    [length, dim] = inpX_T.shape
    
    print('Training des Netzes mit ' + str(length) + " Lehrersignalen über " + str(epoch) + " Epochen")
    
    [w1, w2, b1, b2] = initialize_weights(inpDim, hiddenNeurons, dim - inpDim)
    errorList = []
    
    for e in range (epoch):
    #for e in range(1,2):
        inpPerm = np.random.permutation(inpX_T)
        for i in range (0, length):
        #for i in range (0, 1):
            samplePoint = inpPerm[[i],:]
            #print(samplePoint)
            inpX = samplePoint[:, (0,1)]
            T = np.asscalar(samplePoint[:, 2])

            [u1, y1, y2] = forward(inpX, w1, w2, b1, b2)
            [d1, d2] = prop_error(T, y2, w2, tanh_deriv(u1))

            #print("y1: " + str(y1.shape))
            #print("w1: " + str(w1.shape))
            #print("d1: " + str(d1.shape))
            #print("d2: " + str(d2.shape))
            #print("x: " + str(samplePoint.shape))
            #print("b1: " + str(b1.shape))
            #print("b2: " + str(b2.shape))

            #Lernschritt
            w2 = np.add(w2, lernRate*y1.transpose()*d2)
            w1 = np.add(w1, lernRate*inpX.transpose()*d1)

            b2 = b2 + lernRate*d2
            b1 = b1 + lernRate*d1


        ## Netzwerkfehler berechnen
        error = 0
        for i in range (0, length):
            samplePoint = inpPerm[[i],:]
            inpX = samplePoint[:, (0,1)]
            T = np.asscalar(samplePoint[:, 2])

            [u1, y1, y2] = forward(inpX, w1, w2, b1, b2)

            #print("T: " + str(T))
            #print("Y2: " + str(y2))

            error += (T-y2)**2

        #print("ERROR: " + str(error))
        errorList.append(error)
    return [errorList, w1, w2, b1, b2]

### Initialiserung der Parameter und Training:
- X und Y entsprechen dem Datensatz
- Z ist das Lehrersignal

In [112]:
############
## Generate some sample data
def f(x,y):
    return np.sin(np.sqrt(x**2 + y**2)) + np.cos(.9*(x-y))

X = np.linspace(-6, 6, 30)
Y = np.linspace(-6, 6, 30)
x, y = np.meshgrid(X, Y)
z = f(x, y)

# Change the shape of the data point
inpX_T = np.reshape(np.dstack([x,y,z]), (900, 3))
#print(outT.shape)
#print(inpX.shape)

############
## Initialize network parameter
inputNeuronen = 2
hiddenNeuronen = 100
lernRate       = 0.01
epochen        = 1000
outputNeuronen = 1;

#############
## Start training of the neural network
[errors, w1, w2, b1, b2] = training(hiddenNeuronen, lernRate, inpX_T, inputNeuronen, epochen)
#print(len(errors))
print("Anfangsfehler: " + str(errors[0]))
print("Endfehler:     " + str(errors[len(errors) - 1]))
#print("Fehlerentwicklung: " + str(errors))

Training des Netzes mit 900 Lehrersignalen über 1000 Epochen
Anfangsfehler: 597.0223765057898
Endfehler:     3.7847749993400206


### Visualisierung der Daten

In [114]:
# Anzuzeigende Daten erzeugen
approx = np.zeros(x.shape)
diff = np.zeros(x.shape)

[len1, len2] = x.shape
for a in range(0, len1):
    for b in range(0, len2):
        inpX = [x[a][b], y[a][b]];
        #print(inpX)
        [_, _, y2] = forward(inpX, w1, w2, b1, b2)
        approx[a][b] = y2
        diff[a][b] = f(x[a][b], y[a][b]) - y2


# Daten visualisieren
fig = plt.figure()
ax1 = fig.add_subplot(2,2,1, projection='3d')
surf1 = ax1.plot_surface(x,y,z, cmap = 'plasma',
                       linewidth=0, antialiased=False)
ax1.set_title('Original')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('Original');

ax2 = fig.add_subplot(2,2,2, projection='3d')
surf2 = ax2.plot_surface(x,y,approx, cmap = 'plasma',
                       linewidth=0, antialiased=False)
ax2.set_title('Approximation')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('Approximation');

ax3 = fig.add_subplot(2,2,3, projection='3d')
surf3 = ax3.plot_surface(x,y,diff, cmap = 'plasma',
                       linewidth=0, antialiased=False)
ax3.set_title('Differenz')
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.set_zlabel('Differenz');

plt.subplot(2,2,4)
plt.title("Quadratischer Fehler über die Epochen")
plt.xlabel('Epoche')
plt.ylabel('Fehler')
plt.plot(np.arange(0, len(errors), 1), errors)

plt.tight_layout()

FigureCanvasNbAgg()